# **Chargement des bibliothèques et fonctions: à executer**

In [ ]:
## pas besoins de lancer ce code si vous ne travaillez pas avec google colab et si vos fichiers ne sont pas stockés dans votre drive Google
## instalation de google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### Pour la traduction en anglais: installation des packages pour pouvoir détecter les langues et faire la traduction !!
!pip install googletrans==4.0.0-rc1
print('googletrans est instalé: ML\n')
!pip install langdetect
print('Langdetect est instalé : ML\n')

googletrans est instalé: ML

Langdetect est instalé : ML



In [ ]:
## à lancer pour importer les bibliothèques nécessaires. (selenieum ne fonctionne pas sur le web)
import warnings
warnings.filterwarnings("ignore")
from time import sleep
# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.common.keys import Keys
import pandas as pd
import re
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import datetime
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
### fonction visulalisation d'un avis cliens : choisir un indice

def afficher (df,indice):
# indice = 5
  print('Valeurs pour indice',indice)
  for colonne in df.columns:
    print(colonne , ':------>',df[colonne][indice],'')

# **Feature engineering: emojis, detection langue, ajout colonne commentaire_text .**

**Data cleaning : ajout des colonnes: emojis, **

In [ ]:
### chargement et lecture de fichier brut non traité
df_brut = pd.read_excel('/content/drive/MyDrive/Projet Datascientest/Datas/Final_data_scraped_traité_non_traduit.xlsx')


In [ ]:
df = df_brut
df = df.drop(columns=['Unnamed: 0'])
df.info()
# Final_data_scraped_traité_non_traduit.xlsx

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65584 entries, 0 to 65583
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   categorie_bis          65584 non-null  object        
 1   companies              65584 non-null  object        
 2   noms                   65580 non-null  object        
 3   titre_com              65584 non-null  object        
 4   commentaire            65584 non-null  object        
 5   reponses               17824 non-null  object        
 6   notes                  65584 non-null  int64         
 7   date_experience        65584 non-null  datetime64[ns]
 8   date_commentaire       65584 non-null  datetime64[ns]
 9   site                   65584 non-null  object        
 10  nombre_pages           65584 non-null  int64         
 11  date_scrap             65584 non-null  object        
 12  verified               65584 non-null  int64         
 13  a

In [ ]:
afficher(1101)

Valeurs pour indice 1101
categorie_bis :------> furniture_store 
companies :------> Vente-unique.com  
noms :------> Fanta Kourouma 
titre_com :------> J’adore ma table facile à monter et le… 
commentaire :------> J’adore ma table facile à monter et le design est juste magnifique 🥰😍✌🏽Merci merci merci !!! 
reponses :------> nan 
notes :------> 5 
date_experience :------> 2022-04-30 00:00:00 
date_commentaire :------> 2022-04-30 00:00:00 
site :------> https://www.trustpilot.com/review/www.vente-unique.com?page=4 
nombre_pages :------> 27 
date_scrap :------> 17-09-2023 
verified :------> 0 
année_experience :------> 2022 
mois_experience :------> 4 
jour_experience :------> 30 
année_commentaire :------> 2022 
mois_commentaire :------> 4 
jour_commentaire :------> 30 
leadtime_com_exp :------> 0 
nombre_caractères :------> 92 
nombre_maj :------> 2 
nombre_car_spé :------> 8 
caractères_spé :------> ['’', '🥰', '😍', '✌', '🏽'] 
emojis_positifs_count :------> 2 
emojis_negatifs_count :---

In [ ]:
!pip install emoji
import emoji
# df.drop('nombre_car_spé2', axis =1)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 5.5 MB/s eta 0:00:00


In [ ]:
# Comptez le nombre total de caractères

df['nombre_caractères'] = df['commentaire'].apply(len)

# Comptez le nombre de caractères en majuscules
df['nombre_maj'] = df['commentaire'].apply(lambda x: sum(1 for c in x if c.isupper()))

# Comptez le nombre de caractères spéciaux (en excluant les lettres et les chiffres)
# df['nombre_car_spé'] = df['commentaire'].apply(lambda x: len(re.sub('[a-zA-Z0-9]', '', x)))
df['nombre_car_spé'] = df['commentaire'].apply(lambda x: len([c for c in x if not c.isalnum() and not c.isspace()]))
df['caractères_spé'] = df['commentaire'].apply(lambda x: [c for c in x if not c.isalnum() and not c.isspace() and c !='.' and c !=',' and c !='?' and c !='!'])

# Comptez le nombre de smileys (en utilisant un exemple simple de smiley ":)")
# df['nombre_smiley_p'] = df['commentaire'].apply(lambda x: x.count('😊')+x.count('👍')+x.count('😊'))

# Définissez une liste d'emojis positifs que vous souhaitez compter
emojis_positifs = ['😀', '😁', '😂', '🤣', '😃', '😄', '😅', '😆', '😇', '😉', '😊', '😋', '😌', '😍', '😎', '😏', '😐', '😑','👍', '👏', '🙌', '🤝', '🙏', '✌️','✌', '🤞', '🤟', '🤘', '🤙', '👌', '👈', '👉', '👆', '👇', '☝️', '✋', '🤚', '🖐️', '🖖','👋', '🤗', '🤩','💖','💓', '💕', '💞', '💘' ,'💗' ,'💝','❤️','🧡' ,'💛' ,'💚', '💙' ,'💜' ,'🤎', '🖤','❤' ,'🤍', '💟', '💫', '💯']

# Fonction pour compter les emojis positifs dans une chaîne
def compter_emojis_positifs(texte):
    count = 0
    for char in texte:
        if char in emojis_positifs:
            count += 1
    return count

# Définissez une liste d'emojis positifs que vous souhaitez compter
emojis_negatifs = ['😔', '😕', '😖', '😣', '😢', '😥', '😰', '😨', '😩', '😫', '😤', '😡', '😠', '😈', '👿', '💀', '☠️', '💩', '🤡','👎','👊', '🖕','💔']

# Fonction pour compter les emojis positifs dans une chaîne
def compter_emojis_negatifs(texte):
    count = 0
    for char in texte:
        if char in emojis_negatifs:
            count += 1
    return count

# Appliquez la fonction à la colonne du dataframe
df['emojis_positifs_count'] = df['caractères_spé'].apply(compter_emojis_positifs)
df['emojis_negatifs_count'] = df['caractères_spé'].apply(compter_emojis_negatifs)




In [ ]:
df_emoji=df.loc[(df['emojis_negatifs_count']>0) | (df['emojis_positifs_count']>0) ]

afficher(6548)
# df_emoji.info()
df_emoji.head()
# print(df['emojis_positifs_count'].unique())
# print(df['emojis_negatifs_count'].unique())

Valeurs pour indice 6548
categorie_bis :
-----------> travel_insurance_company 
companies :
-----------> Allianz Travel   
noms :
-----------> Guanliang Pan 
titre_com :
-----------> Please avoid 
commentaire :
-----------> The Visa centre told me that the travel insurance is not good enough and I need to get a new one that specified covers the whole of Europe instead of specifying 1 country. I wanted to get a confirmation or clarification for Allianz but no response for 3 weeks and I had to buy new travel insurance so I won't miss my 2nd appointment for Visa. Also, no refund as it passed 14 days.Not expensive, just annoying and wasted so much of my time. 
reponses :
-----------> nan 
notes :
-----------> 1 
date_experience :
-----------> 2023-06-01 00:00:00 
date_commentaire :
-----------> 2023-06-02 00:00:00 
site :
-----------> https://www.trustpilot.com/review/allianz-voyage.fr?page=1 
nombre_pages :
-----------> 2 
date_scrap :
-----------> 17-09-2023 
verified :
-----------> 0 
a

,categorie_bis,companies,noms,titre_com,commentaire,reponses,notes,date_experience,date_commentaire,site,...,année_commentaire,mois_commentaire,jour_commentaire,leadtime_com_exp,nombre_caractères,nombre_maj,nombre_car_spé,caractères_spé,emojis_positifs_count,emojis_negatifs_count
5,furniture_store,Made.com,Sobia,Hello Thanks I like all what get from…,Hello Thanks I like all what get from your web...,"Hello ,Thank you for your order and your 5 sta...",5,2022-09-16,2022-09-16,https://www.trustpilot.com/review/made.com/fr?...,...,2022,9,16,0 days,112,4,9,"[…, ❤, ️, 👍, 🏼]",2,0
56,furniture_store,Made.com,Sido,Produit conforme à la description 👍,Produit conforme à la description 👍,Bonjour Sidonie;Milles merci pour votre avis 5...,5,2022-07-19,2022-07-20,https://www.trustpilot.com/review/made.com/fr?...,...,2022,7,20,1 days,35,1,1,[👍],1,0
67,furniture_store,Made.com,Walter,Emballage nickel envoie nickel rien à…,Emballage nickel envoie nickel rien à redire👍,BonjourMerci pour votre commande et votre avis...,5,2022-02-04,2022-02-04,https://www.trustpilot.com/review/made.com/fr?...,...,2022,2,4,0 days,45,1,1,[👍],1,0
73,furniture_store,Made.com,Massiry,MADE,MADE: The best destination for chic home desig...,BonjourMerci pour votre commande et votre avis...,5,2022-01-04,2022-01-04,https://www.trustpilot.com/review/made.com/fr?...,...,2022,1,4,0 days,55,6,4,"[:, ❤, ️]",1,0
102,furniture_store,Made.com,Pons,Parfait,Parfait! 👌🏼,"Bonjour ,Mille mercis pour votre avis 5 étoile...",5,2022-06-11,2022-06-11,https://www.trustpilot.com/review/made.com/fr?...,...,2022,6,11,0 days,11,1,3,"[👌, 🏼]",1,0




---


**Détection de la langue et préparation des  commentaires à traduire en Anglais.**

In [ ]:
## installation des packages !!
!pip install langdetect
print('Langdetect est instalé : ML\n')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=c90d205a79ddc9f9e1e9c4259f15ff86dd46e5b90a90af12c1d4ed6e60f1a0c5
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
Langdetect est instalé : ML



Avant de commencer la traduction des commentaires, il est nécessaire de supprimer les caractères spéciaux, les émojis dans la colonne à traduire. sans cette opération, vous allez rencontrer des bugs.

In [ ]:
######################################################""""
### création de la colonne commentaire_text = commentaires nettoyés sans les emoji, les caractères spéciaux...
def remove_special_characters_and_emojis(text):
    # Supprime les emojis
    # text = re.sub(r'\\[xX][0-9a-fA-F]+', '', text)

    # Supprime les caractères spéciaux en conservant uniquement les lettres et les espaces
    text = re.sub(r'[^a-zA-Z0-9\sàáâäçèéêëìíîïñòóôöùúûüýÿ\s,.;\']', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['commentaire_text'] = df['commentaire'].apply(remove_special_characters_and_emojis)


In [ ]:
afficher(5)

Valeurs pour indice 5
Unnamed: 0 :
-----------> 5 
categorie_bis :
-----------> furniture_store 
companies :
-----------> Made.com  
noms :
-----------> Sobia 
titre_com :
-----------> Hello Thanks I like all what get from… 
commentaire :
-----------> Hello Thanks I like all what get from your website .. best quality love them all… I keep counting to buy .. ❤️👍🏼 
reponses :
-----------> Hello ,Thank you for your order and your 5 star review, We are delighted that your experience with MADE has been so positive.We hope to see you soon on MADE.COM for more design projectsHave a nice day, SincerelyThe MADE.COM team 
notes :
-----------> 5 
date_experience :
-----------> 2022-09-16 00:00:00 
date_commentaire :
-----------> 2022-09-16 00:00:00 
site :
-----------> https://www.trustpilot.com/review/made.com/fr?page=1 
nombre_pages :
-----------> 53 
date_scrap :
-----------> 17-09-2023 
verified :
-----------> 0 
année_experience :
-----------> 2022 
mois_experience :
-----------> 9 
jour_exp

**Identifier la langue**

In [ ]:
## detecter (identifier) la langue de commentaire et supprimer les lignes avec langue non détectée

from langdetect import detect

# Créez une fonction pour détecter la langue
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'Non détectée'

# Appliquez la fonction à la colonne 'commentaires'
df['langue'] = df['commentaire_text'].apply(detect_language)

print('Langue détectée')

## supprimer les lignes avec langue = Non détectée

critère_sup = (df['langue'] == 'Non détectée')

print('avant sup',df['langue'].count())
df = df.drop(df[critère_sup].index)
print('après sup',df['langue'].count())


Langue détectée
avant sup 65584
après sup 65463


Désormais , nous avons détecté la langue de la grande partie des lignes. cependant, certaines lignes ne contiennent pas de texte. le code permet d'identifier ces lignes en leur affectant le mot: 'Non détectée'.
d'après la première analyse, il s'agit des commentaires non sinificatfs '...' , des émoji, ou une serie de lettres sans aucun sens.
deux solutions sont possibles:
- supprimer ces lignes vue qu'elles ne représentent qu'un petit nombre de lignes
- remplacer le contenu des ces lignes par le titre de commentaire s'il est présent.

la moyenne des notes de ces lignes est égale à 4.62, ce qui est une bonne note.
on propose de les supprimer!!

In [ ]:
# df.head()
langue_not_detected =df.loc[df['langue'] =='Non détectée']
print('le nombre de lignes avec langue = Non détectée:',langue_not_detected['commentaire'].count(), '/',df['langue'].count())
display(langue_not_detected['commentaire'].unique())
print('\n')
display(langue_not_detected['titre_com'].unique())
print('\n')
display(langue_not_detected['notes'].describe())

le nombre de lignes avec langue = Non détectée: 0 / 65463


array([], dtype=object)

array([], dtype=object)

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: notes, dtype: float64

In [ ]:
### enregistrer le fichier dans un fichier Excel.
df = pd.to_excel('/content/drive/MyDrive/Projet Datascientest/Datas/Final_data_scraped_traité_non_traduit_2.xlsx', index = False)

# **Feature engineering: Traduction des commentaires en Anglais (New methode)**

In [ ]:
### chargement et lecture de fichier Final_data_scraped_traité_non_traduit_2
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Projet Datascientest/Datas/Final_data_scraped_traité_non_traduit_2.xlsx')

In [ ]:
###########################################################################################################################
### nouvelle fonction de traduction bis; elle permet de traduire certaines lignes ( en fonction de la colonne verif_traduction)
###########################################################################################################################

import pandas as pd

from googletrans import Translator
from langdetect import detect
translator = Translator()

## il falait ajouter cette ligne pour que les commentaires en Majiscule soient traduits!!!
df['commentaire_text'] = df['commentaire_text'].str.lower()

# Définissez une fonction "traduire" qui sera appliquée aux lignes où "test_com" est "ERR"
def traduire(mon_df):
    if mon_df['verif_traduction'] == '!!ER!!':
    # if mon_df['langue_bis'] == 'fr':
        try:
            # translated_text = translator.translate(mon_df['commentaire_text'], src = detect(mon_df['commentaire_text']) , dest='en')
            translated_text = translator.translate(mon_df['commentaire_text'], src = 'fr' , dest='en')
            return translated_text.text + '!!OK!!'
        except:
            return mon_df['commentaire_en'] + '!!ER!!'
    else:
        return mon_df['commentaire_en'] + '!!EN!!'


## création de la nouvelle colonne traduite
df['commentaire_en_bis'] = df.apply(traduire, axis=1)

# df['commentaire_en_bis'] = df['commentaire_text'].apply(traduire)

df['verif_traduction'] = df['commentaire_en_bis'].str[-6:]
df['commentaire_en_bis'] = df['commentaire_en_bis'].str[:-6]
##
df['commentaire_en'] = df['commentaire_en_bis']

#### enregistrement de fichier final
df.to_excel('/content/drive/MyDrive/Projet Datascientest/Datas/Final_data_scraped_traité_traduit_v2.xlsx', index = False)
print('Fichier enregistré en V2')

### vérifier et afficher le résultat
print('Répartition par verif_traduction:\n')
print(df["verif_traduction"].value_counts())

# df.head(30)
df_er =df.loc[df['verif_traduction']=='!!ER!!']
df_er.head()

## !!OK!!   : la traduction s'est bien passée normalement
## !!ER!!   : le commentaire n'a pas été traduit erreur6
## !!EN!!   : le commentaires est déja en anglais, donc pas de traduction


Fichier enregistré en V2
Répartition par verif_traduction:

!!EN!!    65452
!!OK!!        8
Name: verif_traduction, dtype: int64
Valeurs pour indice 52810
categorie_bis :
-----------> travel_agency 
companies :
-----------> lastminute.com  
noms :
-----------> miot 
titre_com :
-----------> Bonne prise en charge 
commentaire :
-----------> Bonne prise en charge 
reponses :
-----------> nan 
notes :
-----------> 5 
date_experience :
-----------> 2023-09-05 00:00:00 
date_commentaire :
-----------> 2023-09-10 00:00:00 
site :
-----------> https://www.trustpilot.com/review/www.fr.lastminute.com?page=1 
nombre_pages :
-----------> 47 
date_scrap :
-----------> 23-09-2023 
verified :
-----------> 0 
année_experience :
-----------> 2023 
mois_experience :
-----------> 9 
jour_experience :
-----------> 5 
année_commentaire :
-----------> 2023 
mois_commentaire :
-----------> 9 
jour_commentaire :
-----------> 10 
leadtime_com_exp :
-----------> 5 
nombre_caractères :
-----------> 21 
nombre_m

In [ ]:
# df['commentaire_en']=df['commentaire_en_bis']

print('Répartition par verif_traduction:\n')
print(df["verif_traduction"].value_counts())
# df.head(30)
df_er =df.loc[(df['verif_traduction']=='!!ER!!')]
df_er.head()
# df_er['langue'].value_counts()

Répartition par verif_traduction:

!!EN!!    65452
!!OK!!        8
Name: verif_traduction, dtype: int64


,categorie_bis,companies,noms,titre_com,commentaire,reponses,notes,date_experience,date_commentaire,site,...,nombre_maj,nombre_car_spé,caractères_spé,emojis_positifs_count,emojis_negatifs_count,commentaire_text,langue,commentaire_en,verif_traduction,commentaire_en_bis


In [ ]:
#### enregistrement de fichier final
df.to_excel('/content/drive/MyDrive/Projet Datascientest/Datas/Final_data_scraped_traité_traduit_ok.xlsx', index=False)


# Feature engineering: Regouper les valeurs, Word Cloud

In [ ]:
### chargement et lecture de fichier Final_data_scraped_traité_non_traduit_2
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Projet Datascientest/Datas/Final_data_scraped_traité_traduit_ok.xlsx')

In [ ]:

# Calculez les statistiques descriptives
mean = df['nombre_caractères'].mean()
std = df['nombre_caractères'].std()
Q1 = df['nombre_caractères'].quantile(0.25)
Q3 = df['nombre_caractères'].quantile(0.75)

# Identifiez les valeurs aberrantes en utilisant par exemple la règle des 1,5*IQR
IQR = Q3 - Q1
borne_inf = Q1 - 1.5 * IQR
borne_sup = Q3 + 1.5 * IQR

# données sans les valeurs aberrantes
valeurs_sans_aberrantes = df[(df['nombre_caractères'] >= borne_inf) & (df['nombre_caractères'] <= borne_sup)]
# données avec les valeurs aberrantes
valeurs_aberrantes = df[(df['nombre_caractères'] < borne_inf) | (df['nombre_caractères'] > borne_sup)]


In [ ]:
len(valeurs_aberrantes)

7830

In [ ]:
### chargement et lecture de fichier Final_data_scraped_traité_traduit
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Projet Datascientest/Datas/Final_data_scraped_traité_traduit_ok.xlsx')
# df = df.drop(columns=['Unnamed: 0'])

In [ ]:

# Définissez les intervalles et les étiquettes correspondantes
intervals = [0, 10 , 30 , 50, 100 , float('inf')]  # Les intervalles
labels = [1, 2, 3, 4 , 5]  # Les étiquettes correspondantes
# Utilisez la fonction cut pour créer une nouvelle colonne 'categories'
df['cat_nombre_caractères'] = pd.cut(df['nombre_caractères'], bins=intervals, labels=labels)

#########################

# Définissez les intervalles et les étiquettes correspondantes
intervals = [0, 10, 30, 50, 100, float('inf')]  # Les intervalles
labels = [1, 2, 3, 4, 5]  # Les étiquettes correspondantes
# Utilisez la fonction cut pour créer une nouvelle colonne 'categories'
df['cat_nombre_maj'] = pd.cut(df['nombre_maj'], bins=intervals, labels=labels)

#### regrouper les notes en deux catégories
df['notes_bis'] = df['notes'].replace({1 : 0 , 2 : 0 , 3 : 0, 4 : 1 , 5 : 1}, inplace = False)

print(df['cat_nombre_caractères'].unique())
print(df['cat_nombre_maj'].unique())
print(df['notes_bis'].unique())

[5, 3, 4, 2, 1]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]
[1, 2, NaN, 3, 4, 5]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]
[0 1]


**Identification des Word cloud**

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords

# Téléchargez la liste des stopwords si ce n'est pas déjà fait
nltk.download('stopwords')

# Supprimer les stopwords et créer une nouvelle colonne
stop_words = set(stopwords.words('english'))
df['commentaire_en_sans_SW'] = df['commentaire_en'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Créez une nouvelle colonne "verif_reponses" en fonction de "reponses"
df['verif_reponses'] = df['reponses'].apply(lambda x: 0 if pd.isna(x) else 1)
# compter le nombre de point d'intérogation '?'
df['nombre_point_intero'] = df['commentaire'].apply(lambda x: x.count('?'))
# compter le nombre de point d'exclamation '!'
df['nombre_point_exclam'] = df['commentaire'].apply(lambda x: x.count('!'))

In [ ]:
# enregistrer le dataframe dans un fichier Final_data_scraped_traité_traduit.xlsx

df.to_excel('/content/drive/MyDrive/Projet Datascientest/Datas/Final_data_scraped_traité_traduit_ok_fini.xlsx', index=False)
### dernier modif 14 oct 19:57



---

# Feature engineering: Traduction des commentaires en Anglais (old methode)
**Début de la traduction**

In [ ]:
### chargement et lecture de fichier Final_data_scraped_traité_non_traduit_2
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Projet Datascientest/Datas/Final_data_scraped_traité_non_traduit_2.xlsx')



---



**La traduction des commentaires prends beaucoup de temps. c'est pour cette raison que j'ai crée des parties de mon df pour faire la traduction partie par partie.**

**Ensuite je regroupe les parties traduites dans un seul df.**

In [ ]:
### diviser le df en plusieurs parties pour traduire morceau par morceau

# Définissez le nombre de parties que vous souhaitez créer
nombre_de_parties = 10

# Calculez le nombre de lignes par partie
lignes_par_partie = len(df) // nombre_de_parties

# Initialisez un dictionnaire pour stocker les parties
parties = {}

# Divisez le DataFrame en parties égales et stockez-les dans le dictionnaire
for i in range(nombre_de_parties):
    debut = i * lignes_par_partie
    fin = (i + 1) * lignes_par_partie
    nom_colonne = f'Partie_{i + 1}'
    parties[nom_colonne] = df.iloc[debut:fin].reset_index(drop=True)

# # Accédez à chaque partie par son nom de colonne
# premiere_partie = parties['Partie_1']
print(nombre_de_parties , ' parties crées, chacune contient: ', lignes_par_partie , 'lignes' )

10  parties crées, chacune contient:  6546 lignes


In [ ]:
## traduire les commentaires en anglais
##premières fonctions utilisée. nouvelle fonction existe
from googletrans import Translator
from langdetect import detect
translator = Translator()

### fonction pour traduire du français vers anglais
def translate_autre_to_english(text):
  if detect(text) != 'en':
    try:
      translated_text = translator.translate(str(text), src = detect(text) , dest='en')
      return translated_text.text + '!!OK!!'
    except:
      return text + '!!ER!!'
  else:
    return text + '!!EN!!'

########################################################
# df_test = df.loc[(df['categorie_bis'] == 'bank')].head(100)
# partie_1 = parties['Partie_1']

## parcourir toutes les parties et enregistrer chaque partie traduite dans un fichier excel
for nom_colonne, partie in parties.items():
    print(f"{nom_colonne}: {len(partie)} lignes")
    df_test = parties[nom_colonne]
    df_test['commentaire_text'] = df_test['commentaire_text'].str.lower() ## il falait ajouter cette ligne pour que les commentaires en Majiscule soient traduits!!!
    ## création de la nouvelle colonne traduite
    df_test['commentaire_en'] = df_test['commentaire_text'].apply(translate_autre_to_english)
    ##
    df_test['verif_traduction'] = df_test['commentaire_en'].str[-6:]
    df_test['commentaire_en'] = df_test['commentaire_en'].str[:-6]
    df_test.to_excel('/content/drive/MyDrive/Projet Datascientest/Datas/Translations/Final_data_scraped_traité_traduit_' + nom_colonne + '.xlsx', index = False)

## !!OK!!   : la traduction s'est bien passée normalement
## !!ER!!   : le commentaire n'a pas été traduit erreur6
## !!EN!!   : le commentaires est déja en anglais, donc pas de traduction


Partie_1: 6546 lignes
Partie_2: 6546 lignes
Partie_3: 6546 lignes
Partie_4: 6546 lignes
Partie_5: 6546 lignes
Partie_6: 6546 lignes
Partie_7: 6546 lignes
Partie_8: 6546 lignes
Partie_9: 6546 lignes
Partie_10: 6546 lignes


In [ ]:

## regrouper les fichiers traduits en un seul dataframe (fichier)

df_1= '/content/drive/MyDrive/Projet Datascientest/Datas/Translations/Final_data_scraped_traité_traduit_Partie_1.xlsx'
df_2= '/content/drive/MyDrive/Projet Datascientest/Datas/Translations/Final_data_scraped_traité_traduit_Partie_2.xlsx'
df_3= '/content/drive/MyDrive/Projet Datascientest/Datas/Translations/Final_data_scraped_traité_traduit_Partie_3.xlsx'
df_4= '/content/drive/MyDrive/Projet Datascientest/Datas/Translations/Final_data_scraped_traité_traduit_Partie_4.xlsx'
df_5= '/content/drive/MyDrive/Projet Datascientest/Datas/Translations/Final_data_scraped_traité_traduit_Partie_5.xlsx'
df_6= '/content/drive/MyDrive/Projet Datascientest/Datas/Translations/Final_data_scraped_traité_traduit_Partie_6.xlsx'
df_7= '/content/drive/MyDrive/Projet Datascientest/Datas/Translations/Final_data_scraped_traité_traduit_Partie_7.xlsx'
df_8= '/content/drive/MyDrive/Projet Datascientest/Datas/Translations/Final_data_scraped_traité_traduit_Partie_8.xlsx'
df_9= '/content/drive/MyDrive/Projet Datascientest/Datas/Translations/Final_data_scraped_traité_traduit_Partie_9.xlsx'
df_10= '/content/drive/MyDrive/Projet Datascientest/Datas/Translations/Final_data_scraped_traité_traduit_Partie_10.xlsx'

liste_file = [df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10]
resultat = pd.DataFrame()

for df_val in liste_file:
    df = pd.read_excel(df_val)
    resultat = pd.concat([resultat, df], ignore_index=True)

# resultat = resultat.drop(columns=['Unnamed: 0.1'])

# Enregistrer le dataframe dans un fichier .csv / xlsx
resultat.to_excel('/content/drive/MyDrive/Projet Datascientest/Datas/Final_data_scraped_traité_traduit_v1.xlsx',index = False)




---



In [ ]:
### chargement et lecture de fichier Final_data_scraped_traité_traduit
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Projet Datascientest/Datas/Final_data_scraped_traité_traduit_v1.xlsx')


In [ ]:
# df = df.drop(columns=['Unnamed: 0'])
# df = df.drop(columns=['Unnamed: 0.1'])
print(df.shape)
df.head(5)
# 65463

(65460, 30)


,categorie_bis,companies,noms,titre_com,commentaire,reponses,notes,date_experience,date_commentaire,site,...,nombre_caractères,nombre_maj,nombre_car_spé,caractères_spé,emojis_positifs_count,emojis_negatifs_count,commentaire_text,langue,commentaire_en,verif_traduction
0,furniture_store,Made.com,Dara Teste,I placed an order and paid via Paypal…,I placed an order and paid via Paypal on augus...,NaN,1,2023-03-11,2023-03-11,https://www.trustpilot.com/review/made.com/fr?...,...,119,4,1,[],0,0,i placed an order and paid via paypal on augus...,en,i placed an order and paid via paypal on augus...,!!EN!!
1,furniture_store,Made.com,marie,Suspension correspondant à la…,Suspension correspondant à la description fait...,"Bonjour Marie,Mille mercis pour votre avis 5 é...",5,2022-10-11,2022-10-26,https://www.trustpilot.com/review/made.com/fr?...,...,103,4,3,[],0,0,suspension correspondant à la description fait...,fr,suspension corresponding to the description ma...,!!OK!!
2,furniture_store,Made.com,Bruno,Great products and fantastic service.,Great products and fantastic service.,"Bonjour Bruno,Mille mercis pour votre avis 5 é...",5,2022-10-24,2022-10-24,https://www.trustpilot.com/review/made.com/fr?...,...,37,1,1,[],0,0,great products and fantastic service.,en,great products and fantastic service.,!!EN!!
3,furniture_store,Made.com,Shurong,Fantastic quality,Fantastic quality! Worth waiting. We ordered t...,"Bonjour,Merci pour votre commande et votre avi...",5,2022-06-05,2022-06-05,https://www.trustpilot.com/review/made.com/fr?...,...,325,7,13,['’'],0,0,fantastic quality worth waiting. we ordered th...,en,fantastic quality worth waiting. we ordered th...,!!EN!!
4,furniture_store,Made.com,Amene,color precise,Hello I like made. It's a bit expensive but an...,"Bonjour Amene,Merci pour votre retour d'expéri...",4,2022-08-25,2022-08-25,https://www.trustpilot.com/review/made.com/fr?...,...,344,9,7,"[""'""]",0,0,hello i like made. it's a bit expensive but an...,en,hello i like made. it's a bit expensive but an...,!!EN!!


In [ ]:
## detecter (identifier) la langue de commentaire et supprimer les lignes avec langue non détectée

from langdetect import detect

# Créez une fonction pour détecter la langue
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'Non détectée'

# Appliquez la fonction à la colonne 'commentaires'
df['langue_bis'] = df['commentaire_en'].apply(detect_language)

print('Langue détectée')

## supprimer les lignes avec langue = Non détectée

critère_sup = (df['langue_bis'] == 'Non détectée')

print('avant sup',df['langue_bis'].count())
# df = df.drop(df[critère_sup].index)
# print('après sup',df['langue_bis'].count())

Langue détectée


In [ ]:
print('avant sup',df['langue_bis'].count())
df['langue_bis'].value_counts()

avant sup 65460


en    38336
fr    18843
it     2125
ca     1740
es     1040
ro     1011
sk      303
da      283
so      220
af      217
hr      162
nl      158
cy      135
no      117
pt      109
id       80
sl       79
lv       77
sv       66
tl       63
et       49
fi       44
sq       43
hu       32
cs       29
de       28
lt       25
pl       20
sw       15
vi        8
tr        3
Name: langue_bis, dtype: int64

In [ ]:
# display(df.groupby('langue')['verif_traduction'].count())
# display(pd.crosstab(df['companies'], df['langue'], values=df['verif_traduction'], aggfunc='count', margins=True))

# display(pd.crosstab(df['categorie_bis'], df['langue']))
# display(pd.crosstab(df['langue'], df['verif_traduction']))
# df.head()

print('Répartition par verif_traduction:\n')
df["verif_traduction"].value_counts()

print('Répartition par langue:\n')
print(df["langue"].value_counts().head())
print('Répartition par langue_bis:\n')
print(df["langue_bis"].value_counts().head())
df_er = df.loc[df['verif_traduction']=='!!ER!!']
print('Répartition par notes dans le df_er:\n', df_er["verif_traduction"].value_counts())
# df_er["companies"].value_counts()


Répartition par verif_traduction:

Répartition par langue:

en    39688
fr    16629
ca     2882
it     1061
ro     1009
Name: langue, dtype: int64
Répartition par langue_bis:

en    38380
fr    18860
it     2076
ca     1732
es     1062
Name: langue_bis, dtype: int64
Répartition par notes dans le df_er:
 Series([], Name: verif_traduction, dtype: int64)
